In [ ]:
from rdkit import Chem
import pandas as pd
from sklearn.experimental import enable_iterative_imputer  # Required for IterativeImputer
from sklearn.impute import IterativeImputer


sdf_file = r'C:\Qe-CaDD\Project\QuantumML\tox21_10k_data_all.sdf'
suppl = Chem.SDMolSupplier(sdf_file)

data = []


for mol in suppl:
    if mol is None:  
        continue
    properties = mol.GetPropsAsDict()
    smiles = Chem.MolToSmiles(mol)
    properties['SMILES'] = smiles
    data.append(properties)


df = pd.DataFrame(data)


df.to_csv('tox21.csv', index=False)

In [ ]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer


df = pd.read_csv('tox21.csv')

imputer = IterativeImputer(max_iter=10, random_state=0)


df_filled = pd.DataFrame(imputer.fit_transform(df.select_dtypes(include='number')), columns=df.select_dtypes(include='number').columns)


df_filled = pd.concat([df.drop(columns=df.select_dtypes(include='number').columns), df_filled], axis=1)


for column in df_filled.select_dtypes(include='number').columns:
    df_filled[f'binary_{column}'] = (df_filled[column] > 0.5).astype(int)


df_filled.to_csv('tox21_filled.csv', index=False)


In [ ]:
import pandas as pd


df = pd.read_csv('tox21_filled.csv')

def transform_value(value):
    if isinstance(value, (int, float)):  
        return 1 if value > 0.5 else 0  
    return value  

df_transformed = df.map(transform_value)

df_transformed.to_csv('tox21_transformed.csv', index=False)


